In [1]:
import pandas as pd
import ast

# Load CSVs
df_scheme_mapping = pd.read_csv("Top_Optimized_Schemes_with_LP.csv")
df_recommendations = pd.read_csv("User_Based_Recommendations.csv")


In [2]:
# Standardize column names to avoid casing/spacing issues
df_scheme_mapping.columns = df_scheme_mapping.columns.str.strip().str.lower()
df_recommendations.columns = df_recommendations.columns.str.strip().str.lower()

In [3]:
# Function to safely convert stringified lists into Python lists
def safe_eval(val):
    try:
        return ast.literal_eval(val) if isinstance(val, str) and val.startswith("[") else val
    except (ValueError, SyntaxError):
        return val

In [4]:
# Apply safe_eval where needed
df_scheme_mapping["partner_id"] = df_scheme_mapping["partner_id"].apply(safe_eval)
df_recommendations["recommended_products"] = df_recommendations["recommended_products"].apply(safe_eval)
df_recommendations["similarity_scores"] = df_recommendations["similarity_scores"].apply(safe_eval)


In [5]:
# Combine recommendation and scheme data
results = []
for _, row in df_recommendations.iterrows():
    partner_id = row["partner_id"]
    recommended_products = row["recommended_products"]
    similarity_scores = row["similarity_scores"]

    if not isinstance(recommended_products, list) or not isinstance(similarity_scores, list):
        continue  # Skip rows with invalid list structure

    for product, score in zip(recommended_products, similarity_scores):
        schemes = df_scheme_mapping[df_scheme_mapping["product_id"] == product][["scheme_1", "scheme_2", "scheme_3"]]
        
        if not schemes.empty:
            scheme_1, scheme_2, scheme_3 = schemes.iloc[0].fillna("Not Available").values
        else:
            scheme_1, scheme_2, scheme_3 = "Not Available", "Not Available", "Not Available"
        
        results.append([partner_id, product, score, scheme_1, scheme_2, scheme_3])

In [6]:
# Save the final output
df_final_schemes = pd.DataFrame(results, columns=[
    "Partner_id", "Product_id", "Similarity_Scores", "Scheme_1", "Scheme_2", "Scheme_3"
])
df_final_schemes.to_csv("Final_Partner_Product_Schemes.csv", index=False)

print("Final Partner Product Schemes saved successfully.")


Final Partner Product Schemes saved successfully.
